# Лекция 7: Сложность Модели, Переобучение и Методы Борьбы с ним

**Цели лекции:**
1.  Изучить полиномиальную регрессию как способ моделирования нелинейных зависимостей.
2.  Глубоко разобраться в центральной проблеме ML: компромиссе между смещением и дисперсией (bias-variance tradeoff), и ее проявлениях — недообучении и переобучении.
3.  Освоить инструменты диагностики моделей: визуальный анализ кривых обучения и остатков.
4.  Изучить кросс-валидацию как надежный метод оценки обобщающей способности модели.
5.  Понять идею регуляризации (L1 и L2) как метода борьбы с переобучением путем контроля сложности модели.

## 1. Полиномиальная регрессия: Когда линейности недостаточно

На прошлой лекции мы изучили линейную регрессию. Но что если зависимость в данных нелинейная? Простая прямая линия не сможет хорошо описать сложные данные.

In [ ]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

# Сгенерируем нелинейные (квадратичные) данные
np.random.seed(0)
m = 100
X_poly_data = 6 * np.random.rand(m, 1) - 3
y_poly_data = 0.5 * X_poly_data**2 + X_poly_data + 2 + np.random.randn(m, 1)

# Попробуем описать их простой линейной регрессией
plain_lin_reg = LinearRegression()
plain_lin_reg.fit(X_poly_data, y_poly_data)

plt.scatter(X_poly_data, y_poly_data, alpha=0.7, label='Исходные данные')
plt.plot(X_poly_data, plain_lin_reg.predict(X_poly_data), color='red', linewidth=2, label='Простая линейная регрессия')
plt.title('Ограничения простой линейной регрессии')
plt.xlabel('Признак X')
plt.ylabel('Целевая переменная y')
plt.legend()
plt.show()

### 1.1. Идея: Усложняем модель через признаки

Мы можем заставить линейную модель описывать кривую, если **искусственно создадим новые признаки** из существующих. Этот подход называется **Полиномиальная регрессия**.

Мы "обманываем" модель, подавая ей на вход не только исходный признак $x_1$, но и его степени ($x_1^2, x_1^3, ...$). Уравнение линейной регрессии для признаков $x_1$ и $x_2 = x_1^2$ будет выглядеть так:

$$ \hat{y} = w_0 + w_1x_1 + w_2x_2 $$

Если подставить обратно $x_2 = x_1^2$, мы получим:

$$ \hat{y} = w_0 + w_1x_1 + w_2x_1^2 $$

Это уравнение параболы! Для самой модели линейной регрессии ничего не изменилось: она по-прежнему ищет *линейную* комбинацию признаков. Но благодаря тому, что мы преобразовали признаки, итоговая функция становится нелинейной.

### 1.2. Взаимодействие признаков (Interaction Terms)

Когда у нас несколько исходных признаков (например, $x_1$ и $x_2$), генератор полиномиальных признаков создает не только их степени ($x_1^2, x_2^2, ...$), но и их **произведения ($x_1x_2, ...$)**. Их смысл — уловить **синергетический эффект**, когда влияние одного признака зависит от значения другого.

**Пример из жизни 1: Рекламные кампании**

Представим, что мы предсказываем продажи (`y`) на основе бюджета на рекламу в Instagram ($x_1$) и на уличных баннерах ($x_2$).
Простая модель $y = w_0 + w_1x_1 + w_2x_2$ предполагает, что каждый канал вносит независимый вклад. Но что, если человек увидел рекламу в Instagram, а потом баннер на улице "напомнил" ему о товаре и **усилил** эффект? Этот совместный эффект моделируется слагаемым $w_3(x_1x_2)$.

**Пример из жизни 2: Сельское хозяйство**

Предсказываем урожайность (`y`) на основе количества солнечных дней ($x_1$) и количества осадков ($x_2$). Огромное количество солнца без дождей (засуха) или постоянные дожди без солнца (гниение) — плохо. Наилучший урожай будет при **балансе** этих факторов, который как раз и улавливает признак взаимодействия $x_1 \cdot x_2$.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler


# Создаем конвейер (pipeline): сначала добавляем полиномиальные признаки, затем обучаем линейную регрессию
poly_reg_model = make_pipeline(PolynomialFeatures(degree=2, include_bias=False), 
                               LinearRegression())

# Обучаем модель
poly_reg_model.fit(X_poly_data, y_poly_data)

# Готовим данные для построения гладкой кривой
X_plot = np.linspace(-3, 3, 100).reshape(-1, 1)
y_plot = poly_reg_model.predict(X_plot)

# Визуализируем результат
plt.scatter(X_poly_data, y_poly_data, alpha=0.7, label='Исходные данные')
plt.plot(X_plot, y_plot, color='red', linewidth=2, label='Полиномиальная регрессия (2 степень)')
plt.title('Результат работы полиномиальной регрессии')
plt.xlabel('Признак X')
plt.ylabel('Целевая переменная y')
plt.legend()
plt.show()

## 2. Проблема переобучения и недообучения

Полиномиальные признаки — это мощный инструмент. Однако он порождает новый важный вопрос: **а где остановиться?** Какую степень полинома выбрать? Слишком большая сложность приведет к **переобучению**.

### 2.1. Смещение (Bias) и Дисперсия (Variance)

Это центральная проблема в машинном обучении.

*   **Недообучение (Underfitting, High Bias):** Модель слишком проста и не способна уловить основные закономерности в данных. Она будет показывать **плохое качество и на обучающих, и на новых данных**.
*   **Переобучение (Overfitting, High Variance):** Модель слишком сложна. Она не просто выучила закономерности, но и "запомнила" случайный шум обучающей выборки. Она будет показывать **отличное качество на обучающих данных, но очень плохое на новых**.

**Золотая середина:** Нам нужна модель, которая хорошо обобщает закономерности и будет работать одинаково хорошо как на старых, так и на новых данных. Это называется **хорошей обобщающей способностью**.

Визуально это можно представить так:

<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-11.png" alt="Хорошая модель" width="400"></td>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-1.png" alt="Переобученная модель" width="400"></td>
  </tr>
</table>

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(14, 8))
degrees = [1, 2, 20]
for i, degree in enumerate(degrees):
    ax = plt.subplot(1, len(degrees), i + 1)
    
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_poly_data, y_poly_data)
    
    X_plot = np.linspace(-3, 3, 100).reshape(-1, 1)
    y_plot = model.predict(X_plot)
    
    plt.scatter(X_poly_data, y_poly_data, alpha=0.5)
    plt.plot(X_plot, y_plot, color='red', linewidth=2)
    plt.title(f'Степень полинома = {degree}')
    plt.xlabel('X')
    plt.ylabel('y')
    plt.ylim(0, 10)
    
    if degree == 1:
        plt.text(0.5, 8, "Недообучение (High Bias)", fontsize=12)
    if degree == 2:
        plt.text(0, 8, "Оптимальная модель", fontsize=12)
    if degree == 20:
        plt.text(-2.5, 1, "Переобучение (High Variance)", fontsize=12)

plt.tight_layout()
plt.show()

## 3. Диагностика и Оценка Модели

**Главная проблема:** Как понять, что модель переобучилась, и найти оптимальную сложность?

**Ключевой принцип:** Никогда не оценивать финальное качество модели на тех же данных, на которых она обучалась. Для этого данные делят на части.

### 3.1. Разбиение данных: Train / Validation / Test

*   **Обучающая выборка (train set, ~60-80%):** Используется для обучения модели (подбора весов `w`).
*   **Валидационная выборка (validation set, ~10-20%):** Используется для **выбора модели и ее гиперпараметров** (например, оптимальной степени полинома). Мы выбираем ту модель, которая показывает наименьшую ошибку на *валидационных* данных.
*   **Тестовая выборка (test set, ~10-20%):** "Неприкосновенный запас". Используется **только один раз** для финальной, честной оценки качества выбранной модели.

Поведение ошибок на обучающей и тестовой выборках в зависимости от сложности модели — это классический способ диагностики.

![Кривые обучения](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-2.png)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

X_train, X_val, y_train, y_val = train_test_split(X_poly_data, y_poly_data, test_size=0.3, random_state=10)

train_errors, val_errors = [], []
degrees = range(1, 15)

for degree in degrees:
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_train, y_train)
    
    y_train_predict = model.predict(X_train)
    y_val_predict = model.predict(X_val)
    
    train_errors.append(np.sqrt(mean_squared_error(y_train, y_train_predict)))
    val_errors.append(np.sqrt(mean_squared_error(y_val, y_val_predict)))

plt.figure(figsize=(8, 5))
plt.plot(degrees, train_errors, 'r-+', linewidth=2, label='Ошибка на обучении (Train)')
plt.plot(degrees, val_errors, 'b-', linewidth=3, label='Ошибка на валидации (Validation)')
plt.legend(loc='upper right', fontsize=14)
plt.xlabel('Сложность модели (степень полинома)', fontsize=14)
plt.ylabel('RMSE', fontsize=14)
plt.title('Кривые обучения', fontsize=16)
plt.grid(True)
plt.tight_layout()
plt.show()

**Вывод из графика (Кривые обучения):**
*   Ошибка на **обучении** постоянно падает с ростом сложности — модель все лучше подстраивается под данные, которые видит.
*   Ошибка на **валидации** сначала падает, достигает минимума (в нашем случае при степени 2-3), а затем начинает расти. Это точка, где модель начинает переобучаться.

**Наша цель — найти модель со сложностью, соответствующей минимуму на валидационной кривой.**

### 3.2. Визуальный анализ остатков

Остатки ($y_{True} - y_{pred}$) — это ошибки нашей модели. В идеале, они должны быть случайным шумом без закономерностей.

**Что мы ищем на графике остатков:**
1.  **Случайное распределение:** Точки хаотично разбросаны вокруг нулевой линии.
2.  **Отсутствие паттернов:** Если видна структура (например, парабола), значит, модель не уловила какую-то зависимость.
3.  **Гомоскедастичность:** Разброс точек должен быть одинаковым по всей оси X. Если разброс увеличивается (форма воронки), это называется гетероскедастичностью.

In [ ]:
# Возьмем модель с оптимальной степенью = 2
import seaborn as sns
optimal_model = make_pipeline(PolynomialFeatures(2), LinearRegression())
optimal_model.fit(X_train, y_train)
y_val_pred = optimal_model.predict(X_val)
residuals = y_val.flatten() - y_val_pred.flatten()

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_val_pred.flatten(), y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('График остатков (Residual Plot)')
plt.xlabel('Предсказанные значения')
plt.ylabel('Остатки (ошибки)')
plt.grid(True)
plt.show()

Анализ остатков помогает обнаружить проблемы, которые не видны по одним только метрикам. Классический пример — **квартет Энскомба**: четыре набора данных с одинаковыми статистиками и одинаковой линией регрессии, но совершенно разной структурой.

![Квартет Энскомба](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-3.png)

Только для первого набора данных (вверху слева) линейная регрессия является адекватной моделью. В остальных случаях анализ остатков показал бы явные проблемы.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import seaborn as sns

# Возьмем модель с оптимальной степенью = 2
optimal_model = make_pipeline(PolynomialFeatures(2), LinearRegression())
optimal_model.fit(X_train, y_train)
y_val_pred = optimal_model.predict(X_val)
residuals = y_val.flatten() - y_val_pred.flatten()

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_val_pred.flatten(), y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('График остатков (Residual Plot)')
plt.xlabel('Предсказанные значения')
plt.ylabel('Остатки (ошибки)')
plt.grid(True)
plt.show()

### 3.3. Метод кросс-валидации (Cross-Validation)

Проблема обычного Train/Validation Split в том, что оценка сильно зависит от случайного разделения. **K-Fold Cross-Validation** — это более надежный метод оценки:

1.  Обучающая выборка разбивается на K непересекающихся частей ("фолдов"), например, K=5.
2.  Запускается цикл K раз. На каждой итерации `i`:
    *   Блок `i` используется как **валидационный**.
    *   Оставшиеся K-1 блоков используются для **обучения**.
    *   Оценивается качество на валидационном блоке `i`.
3.  Усредняем K полученных оценок качества.

#### Общий процесс настройки и оценки

```mermaid
graph LR
    A["Данные<br/>X и y"] --> B["Обучающий (+ Валидационный)<br/>набор данных"]
    A --> C["Тестовый<br/>(отложенный) набор"]
    
    subgraph Итеративный цикл настройки
        B --> D["Обучение<br/>модели на Train"]
        D --> F["Оценка<br/>на Validation"]
        F -->|Результат оценки| E["Настройка<br/>гиперпараметров"]
        E --> D
    end
    
    C -->|Финальная проверка| G["Оценка лучшей<br/>модели на Test"]
    E -- лучшая модель --> G
    G --> H["Внедрение<br/>модели"]
```

![K-Fold CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

**Преимущества:**
*   **Надежность:** Оценка меньше зависит от случайности разбиения.
*   **Эффективное использование данных:** Каждый объект побывает в роли тестового ровно один раз.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Данные X_poly_data и y_poly_data должны быть определены ранее в ноутбуке,
# как в предыдущих ячейках. Для полноты примера, определим их здесь снова.
np.random.seed(0)
m = 100
X_poly_data = 6 * np.random.rand(m, 1) - 3
y_poly_data = 0.5 * X_poly_data**2 + X_poly_data + 2 + np.random.randn(m, 1)


# Возьмем нашу оптимальную модель 2-й степени
model_deg_2 = make_pipeline(PolynomialFeatures(2), LinearRegression())

# Запускаем 10-fold кросс-валидацию. 
# cv=10 означает, что данные будут разбиты на 10 частей.
# scoring='neg_mean_squared_error' вычисляет отрицательный MSE.
scores = cross_val_score(model_deg_2, X_poly_data, y_poly_data, cv=10, scoring='neg_mean_squared_error')

# Преобразуем отрицательный MSE в положительный RMSE
rmse_scores = np.sqrt(-scores)

print("Ошибки (RMSE) на каждом из 10 фолдов:")
print(rmse_scores.round(2))
print("\nСредний RMSE на кросс-валидации: {:.2f}".format(rmse_scores.mean()))
print("Стандартное отклонение RMSE: {:.2f}".format(rmse_scores.std()))

## 4. Регуляризация: Борьба с переобучением

Переобученная модель часто имеет очень большие по модулю веса $w$. Она становится слишком "нервной", чувствительной к малейшим изменениям во входных данных.

**Идея регуляризации:** Добавить в функцию потерь **штраф за большие веса**. Модель теперь будет вынуждена искать компромисс: хорошо описывать данные и при этом сохранять веса небольшими.

$$ L_{new}(w) = L_{old}(w) + \alpha \cdot P(w) = \text{MSE} + \text{штраф} $$ 

*   $P(w)$ — **регуляризационный член**.
*   $\alpha \ge 0$ — **коэффициент регуляризации**, гиперпараметр, контролирующий силу штрафа.

### 4.1. L2-регуляризация (Ridge Regression, Гребневая регрессия)

В качестве штрафа используется L2-норма вектора весов (сумма их квадратов).
$$ L_{Ridge}(w) = \sum_{i=1}^{n}(y_i - w^T x_i)^2 + \alpha \sum_{j=1}^{m} w_j^2 $$ 
(свободный член $w_0$ обычно не регуляризуется)

*   **Эффект:** "Сжимает" все веса к нулю, но, как правило, **не обнуляя их полностью**.
*   **Когда полезна:** Хорошо работает, когда большинство признаков вносят свой вклад в результат.

### 4.2. L1-регуляризация (Lasso Regression, Лассо)

В качестве штрафа используется L1-норма вектора весов (сумма их модулей).
$$ L_{Lasso}(w) = \sum_{i=1}^{n}(y_i - w^T x_i)^2 + \alpha \sum_{j=1}^{m} |w_j| $$ 

*   **Эффект:** Может **полностью обнулять веса** некоторых наименее важных признаков.
*   **Ключевое свойство:** Производит **автоматический отбор признаков (feature selection)**, делая модель более простой и интерпретируемой.
*   **Когда полезна:** Когда мы предполагаем, что многие признаки являются "мусорными".

Геометрическая интерпретация помогает понять, почему L1-регуляризация обнуляет веса, а L2 — нет. Область допустимых значений весов для L1 — это ромб, а для L2 — круг. Линии уровня функции потерь (эллипсы) с большей вероятностью коснутся ромба в одной из его вершин (где один из коэффициентов равен нулю).

![Геометрическая интерпретация L1 и L2 регуляризации](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-5.png)

Давайте визуально посмотрим на эффект. Обучим 3 модели на полиномиальных данных 10-й степени (которые склонны к переобучению): простую, Ridge и Lasso.

**Важно:** Перед применением регуляризации **необходимо масштабировать признаки** (например, с помощью `StandardScaler`), иначе штраф будет применяться некорректно.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Создаем конвейер: масштабирование, полиномиальные признаки, модель
pipe_lr = make_pipeline(StandardScaler(), PolynomialFeatures(degree=10, include_bias=False), LinearRegression())
pipe_ridge = make_pipeline(StandardScaler(), PolynomialFeatures(degree=10, include_bias=False), Ridge(alpha=1))
pipe_lasso = make_pipeline(StandardScaler(), PolynomialFeatures(degree=10, include_bias=False), Lasso(alpha=0.1))

# Обучаем
pipe_lr.fit(X_train, y_train)
pipe_ridge.fit(X_train, y_train)
pipe_lasso.fit(X_train, y_train)

# Извлекаем коэффициенты
coeffs_df = pd.DataFrame({
    'Linear Regression': pipe_lr.named_steps['linearregression'].coef_.flatten(),
    'Ridge (alpha=1)': pipe_ridge.named_steps['ridge'].coef_.flatten(),
    'Lasso (alpha=0.1)': pipe_lasso.named_steps['lasso'].coef_.flatten()
})

coeffs_df.plot(kind='bar', figsize=(15, 7))
plt.title('Влияние регуляризации на коэффициенты модели (степень=10)')
plt.ylabel('Значение коэффициента')
plt.ylim(-5, 5) # Ограничим ось Y для наглядности
plt.show()

**Вывод:**
*   **Linear Regression:** Коэффициенты имеют огромные значения (здесь мы их ограничили для наглядности), что является явным признаком переобучения.
*   **Ridge:** Все коэффициенты стали значительно меньше. Модель стала более стабильной.
*   **Lasso:** Большинство коэффициентов стали равны нулю. Модель произвела отбор наиболее важных признаков.

### 4.3. ElasticNet и подбор гиперпараметра `alpha`

*   **ElasticNet** — это комбинация L1 и L2 регуляризаций. Хорошо работает, когда есть группы сильно скоррелированных признаков.
*   **Подбор `alpha`:** Оптимальное значение коэффициента регуляризации $\alpha$ подбирается с помощью **кросс-валидации**. Мы просто перебираем несколько значений (например, `[0.01, 0.1, 1, 10]`) и выбираем то, которое дает наилучшее среднее качество на кросс-валидации.

## 5. Продвинутые темы: Функции Потерь и Методы Оптимизации

В первой лекции мы познакомились с MSE как стандартной функцией потерь. Теперь, понимая контекст, рассмотрим альтернативы.

#### 5.1. Альтернативные Функции Потерь

Выбор функции потерь зависит от свойств данных и целей модели.

*   **MSE (L2 Loss):** $(y_i - \hat{y}_i)^2$
    *   **Свойство:** Сильно штрафует за большие ошибки. **Чувствительна к выбросам**.

*   **MAE (L1 Loss):** $|y_i - \hat{y}_i|$
    *   **Свойство:** Штрафует за ошибки линейно. Более **устойчива (робастна) к выбросам**.

*   **Потеря Хьюбера (Huber Loss):** Гибрид L1 и L2.
    *   **Свойство:** Ведет себя как MSE для маленьких ошибок и как MAE для больших. Сочетает стабильность MSE и устойчивость к выбросам MAE.

#### 5.2. Методы Оптимизации: Как найти минимум?

*   **Аналитическое решение (Нормальное уравнение):**
    $$ w = (X^T X)^{-1} X^T y $$
    *   **Плюсы:** Точное решение без итераций.
    *   **Минусы:** Вычислительно дорого ($O(m^3)$) при большом количестве признаков. Неприменимо ко многим моделям (например, Lasso).

*   **Численное решение (Градиентный спуск):**
    *   **Интуиция:** Итеративные шаги в направлении самого крутого спуска (антиградиента) функции потерь: $w := w - \alpha \cdot ∇L(w)$.

    ![Аналогия с градиентным спуском](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-6.png)
    ![Аналогия с градиентным спуском 1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-8.png)
    ![Аналогия с градиентным спуском 1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-7.png)

    *   **Плюсы:** Универсальный и масштабируемый подход. Основа современного ML.
    *   **Минусы:** Находит приближенное решение, требует подбора скорости обучения $\alpha$.

## Заключение по лекции 7

Сегодня мы рассмотрели ключевые концепции, связанные с обобщающей способностью моделей:
1.  **Полиномиальная регрессия** позволяет описывать нелинейные зависимости, но несет риск **переобучения**.
2.  Мы научились диагностировать переобучение и недообучение с помощью **кривых обучения** и **анализа остатков**.
3.  Для надежной оценки модели и подбора гиперпараметров мы используем **кросс-валидацию**.
4.  **Регуляризация (Ridge, Lasso)** — мощный метод борьбы с переобучением, который штрафует модель за излишнюю сложность (большие веса).
5.  Мы также вернулись к функциям потерь и методам оптимизации, чтобы увидеть, как их выбор влияет на свойства модели, такие как робастность к выбросам.